# **Introduction to Python - Part 2**

**Remember to make a copy of this Notebook in your Google Drive**

First of all make sure that you're connected to an environment. In the top right corner you'll see a 'Connect' button, once you click on it you'll be assigned some resources of disk and ram running in Google Cloud servers.

* Define functions
* Define classes
* Inheritance
* Modules

In [ ]:
import math
import matplotlib.pyplot as plt

## Definition of a function

In [ ]:
def add_numbers(x, y):
    return x+y

In [ ]:
a = 10
b = 4

In [ ]:
add_numbers(a,b)

In [ ]:
def add_numbers(x=10, y=10):
    return x+y

In [ ]:
add_numbers()

In [ ]:
add_numbers(y=2)

In [ ]:
def add_numbers(*args):
    return sum(args)

In [ ]:
add_numbers(123,456,789,987,654,321)

In [ ]:
add_numbers(123,456,789,987,654,321,123,456,789,987,654,321)

In [ ]:
def send_message(channel, **kwargs):
    for k, v in kwargs.items():
        print(f"Sending {channel} on {k} to {v}.")

In [ ]:
send_message("email", monday="Paul, John, Anne", tuesday="Sophie, Kevin, Albert")

In [ ]:
send_message("whatsapp", monday="Paul, John, Anne", tuesday="Sophie, Kevin, Albert", wednesday="Charles, Thomas, Marie")

## Now, let's say we want to control the temperature of an AC.

The following function checks the state of the AC and depending on the state and the current temperature it will increase or decrease it.

In [ ]:
def update_ac(state, current_temp, desired_temp):
    if not state or current_temp == desired_temp:
        return current_temp

    else:
        if current_temp > desired_temp:
            return current_temp - 1
        else:
            return current_temp + 1
    

In [ ]:
update_ac(1, 30, 24)

In [ ]:
timesteps = 10
state = 1
current_temp = 17
desired_temp = 23

for i in range(timesteps):
    print(f"Timestep {i}: Temp: {current_temp}ºC \tTarget: {desired_temp}ºC")
    current_temp = update_ac(state, current_temp, desired_temp)

### Hidden Plot function 

In [ ]:
def plotTemperature(temps, ac, timesteps):
    x = [i for i in range(timesteps)]
    plt.plot(x, temps)
    plt.plot(x, [ac.initial_temp]*len(x), color="r", linewidth="3")
    plt.text(timesteps-150, ac.initial_temp + .5, "Target temperature")
    mn, mx = ac.getMinMax()
    plt.ylim(mn-1.5, mx+1.5)
    plt.xlabel("Time (min)")
    plt.ylabel("Temperature (ºC)")

## Definition of a Class

In [ ]:
class AirConditioner:
    '''
    The init function is the first one called. 
    It allows you to initialize the class with the arguments defined below.
    Some of them are required like 'brand', and some others have a
    default value like 'state', 'temperature', 'target_temp', 'step' and 'delta'.
    '''
    def __init__(self, 
                 brand, 
                 state=0, 
                 temperature=24., 
                 target_temp=None, 
                 step=0.1, 
                 delta = 0.5
            ):
        
        # Attributes of the class
        self.brand = brand
        self.state = state
        self.temperature = temperature
        self.initial_temp = temperature
        self.target_temp = target_temp
        self.step = step
        self.timestep = 0.
        self.delta = delta
        self.mode = None

    '''
    Private function. Defined with a double underscore.
    You cannot access it from an instance of the class.
    '''
    def __increase_temp(self):
        if self.temperature > self.target_temp:
            return
        if self.state:
            self.temperature = self.initial_temp + (self.target_temp - self.initial_temp) * math.exp(-1/(3*self.timestep+0.1))
        else:
            self.temperature = self.timestep / 6 + self.initial_temp

    def __decrease_temp(self):
        if self.temperature < self.target_temp:
            return
        if self.state:
            self.temperature = self.target_temp + (self.initial_temp - self.target_temp) * math.exp(-self.timestep / (self.initial_temp - self.target_temp))
        else:
            self.temperature = -self.timestep / 6 + self.initial_temp
            
    '''
    Public function. As you can see all functions need the variable 'self'
    which allows you call other functions or variables defined in the class.
    '''
    def turn_on(self, target_temp):
        print("Turning AC on...")

        self.state = 1
        self.timestep = 0
        self.target_temp = target_temp
        d = target_temp - self.temperature
        
        if d > 0:
            print("\tActivated heating mode.")
            self.mode = 0
        else:
            print("\tActivated cooling mode.")
            self.mode = 1

        print(f"\tCurrent temperature: {self.temperature}ºC\t-->\tTarget: {self.target_temp}ºC")

    def turn_off(self):
        print("Turning AC off...")
        self.state = 0
        self.timestep = 0.
        self.target_temp = self.initial_temp
        self.initial_temp = self.temperature
        self.mode = not self.mode


    def update_temp(self):
        if self.temperature > self.target_temp and self.mode:
            self.__decrease_temp()

        elif self.temperature < self.target_temp and not self.mode:
            self.__increase_temp()
        
        self.timestep += self.step
        return self.temperature

    def getMinMax(self):
        return (min(self.initial_temp, self.target_temp), max(self.initial_temp, self.target_temp))


In [ ]:
test = AirConditioner("TestBrand")
test.__increase_temp()

In [ ]:
ac = AirConditioner("Hitoshi")
target_temp = 18.
ac.turn_on(target_temp)


temps = []

timesteps = 400

for i in range(timesteps):
    temps.append(round(ac.update_temp(),2))
    if abs(temps[-1] - target_temp) < ac.delta and ac.state:
        print("\tTemperature reached!")
        ac.turn_off()

In [ ]:
plotTemperature(temps, ac, timesteps)

In [ ]:
ac1 = AirConditioner("Hitoshi", temperature=5.)

target_temp = 24.
ac1.turn_on(target_temp)

temps = []

timesteps = 400

for i in range(timesteps):
    temps.append(round(ac1.update_temp(),2))
    if abs(temps[-1] - target_temp) < ac1.delta and ac1.state:
        print("\tTemperature reached!")
        ac1.turn_off()

In [ ]:
plotTemperature(temps, ac1, timesteps)

# Inheritance

In [ ]:
from abc import abstractmethod

In [ ]:
class AirConditioner:
    '''
    The init function is the first one called. 
    It allows you to initialize the class with the arguments defined below.
    Some of them are required like 'owner' or 'brand', and some others have a
    default value like 'color' or 'position'.
    '''
    def __init__(self, 
                 brand, 
                 state=0, 
                 temperature=24., 
                 target_temp=None, 
                 step=0.1, 
                 delta = 0.5
            ):
        
        # Attributes of the class
        self.brand = brand
        self.state = state
        self.temperature = temperature
        self.initial_temp = temperature
        self.target_temp = target_temp
        self.step = step
        self.timestep = 0.
        self.delta = delta
        self.mode = None

    '''
    Private function. Defined with a double underscore.
    You cannot access it from an instance of the class.
    '''

    @abstractmethod
    def increase_temp(self):
        raise NotImplementedError()
    
    @abstractmethod
    def decrease_temp(self):
        raise NotImplementedError()

    '''
    Public function. As you can see all functions need the variable 'self'
    which allows you call other functions or variables defined in the class.
    '''
    def turn_on(self, target_temp):
        print("Turning AC on...")

        self.state = 1
        self.timestep = 0
        self.target_temp = target_temp
        d = target_temp - self.temperature
        
        if d > 0:
            print("\tActivated heating mode.")
            self.mode = 0
        else:
            print("\tActivated cooling mode.")
            self.mode = 1

        print(f"\tCurrent temperature: {self.temperature}ºC\t-->\tTarget: {self.target_temp}ºC")

    def turn_off(self):
        print("Turning AC off...")
        self.state = 0
        self.timestep = 0.
        self.target_temp = self.initial_temp
        self.initial_temp = self.temperature
        self.mode = not self.mode


    def update_temp(self):
        if self.temperature > self.target_temp and self.mode:
            self.decrease_temp()

        elif self.temperature < self.target_temp and not self.mode:
            self.increase_temp()
        
        self.timestep += self.step
        return self.temperature

    def getMinMax(self):
        return (min(self.initial_temp, self.target_temp), max(self.initial_temp, self.target_temp))

In [ ]:
class Hitoshi(AirConditioner):
    def __init__(self, *args):
        super().__init__(*args)

    def increase_temp(self):
        if self.state:
            self.temperature = self.initial_temp + (self.target_temp - self.initial_temp) * math.exp(-1/(3*self.timestep+0.1))
        else:
            self.temperature = self.timestep / 6 + self.initial_temp

    def decrease_temp(self):
        if self.state:
            self.temperature = self.target_temp + (self.initial_temp - self.target_temp) * math.exp(-self.timestep / (self.initial_temp - self.target_temp))
        else:
            self.temperature = -self.timestep / 6 + self.initial_temp       

In [ ]:
class Airoshi(AirConditioner):
    def __init__(self, *args):
        super().__init__(*args)

    def increase_temp(self):
        if self.state:
            self.temperature = self.initial_temp + (self.target_temp - self.initial_temp) * math.exp(-1/(4*self.timestep+0.1))
        else:
            self.temperature = self.timestep / 5 + self.initial_temp

    def decrease_temp(self):
        if self.state:
            self.temperature = self.target_temp + (self.initial_temp - self.target_temp) * math.exp(-self.timestep / 15*(self.initial_temp - self.target_temp))
        else:
            self.temperature = -self.timestep / 5 + self.initial_temp

In [ ]:
ac1 = Hitoshi("Hitoshi")
ac2 = Airoshi("Airoshi")

target_temp = 18.
ac1.turn_on(target_temp)
ac2.turn_on(target_temp)

temps_1 = []
temps_2 = []

timesteps = 100

for i in range(timesteps):
    temps_1.append(round(ac1.update_temp(),2))
    temps_2.append(round(ac2.update_temp(),2))

In [ ]:
plt.plot([i for i in range(len(temps_1))], temps_1)
plt.plot([i for i in range(len(temps_2))], temps_2)

x = [i for i in range(timesteps)]

plt.plot(x, [ac1.target_temp]*len(x), color="r", linewidth="2")
plt.text(0, ac1.target_temp + .3, "Target temperature")
mn, mx = ac1.getMinMax()
plt.ylim(mn-1, mx+1)
plt.xlabel("Time (min)")
plt.ylabel("Temperature (ºC)")
plt.legend(["Hitoshi", "Airoshi"])

# Modules

In [ ]:
import json
from pprint import pprint
from urllib.request import urlopen
with urlopen('https://pypi.org/pypi/sampleproject/json') as resp:
    project_info = json.load(resp)['info']

In [ ]:
type(project_info)

In [ ]:
project_info

In [ ]:
pprint(project_info)

In [ ]:
!git clone https://github.com/aleju/imgaug.git

In [ ]:
img = plt.imread('imgaug/imgaug/quokka.jpg')

In [ ]:
img.shape

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
plt.imshow(img)

In [ ]:
from imgaug.augmenters import blur

In [ ]:
img_blurred = blur.blur_gaussian_(img, 10)

In [ ]:
img_blurred.shape

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
plt.imshow(img_blurred)

In [ ]:
from imgaug.augmenters import EdgeDetect

In [ ]:
aug = EdgeDetect(alpha=0.9)

In [ ]:
img_edge = aug.augment_image(img)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
plt.imshow(img_edge)

# [OPTIONAL 1] 

If you want to practice the concepts shown in this Colab do the following exercise:

- Create an abstract class for your sorting algorithms.
- Implement a couple of sorting algorithms by using the abstract class from the previous step.
- Do not use any external library. Only python code allowed.

# [OPTIONAL 2]

Complete the fibonacci function. This function returns the sum of the two preceding elements in the fibonacci sequence starting from 0 and 1.

In [ ]:
class Fibonacci:
    def fib(self, n):
        ...

In [ ]:
# Tests
f = Fibonacci()
f.fib(0) == 0
f.fib(1) == 1
f.fib(2) == 1
f.fib(3) == 2
f.fib(4) == 3
f.fib(5) == 5
f.fib(6) == 8
f.fib(7) == 13